In [1]:
import numpy as np
import em_algorithm
from simulation_framework.simulate_competency import respondent_population
from simulation_framework.simulate_responses import response_simulation
from scipy.stats import multivariate_normal
import models
import em_algorithm
import pandas as pd
import time

C:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\models


In [2]:
def rmse(y_pred: np.array, y_true: np.array) -> float:
    MSE = np.square(np.subtract(y_pred.flatten(), y_true.flatten())).mean()
    RMSE = np.sqrt(MSE)
    return(float(RMSE))

def experiment_performance(estimated_parameter_dict, real_parameter_dict):
    A_pred = estimated_parameter_dict["item_parameters"]["discrimination_matrix"]
    delta_pred = estimated_parameter_dict["item_parameters"]["intercept_vector"]
    sigma_pred = estimated_parameter_dict["person_parameters"]["covariance"]

    A_true = real_parameter_dict["item_parameters"]["discrimination_matrix"]
    delta_true = real_parameter_dict["item_parameters"]["intercept_vector"]
    sigma_true = real_parameter_dict["person_parameters"]["covariance"]

    print("Absolute diff in A:")
    print(np.abs(A_true-A_pred))

    print("Absolute diff in delta:")
    print(np.abs(delta_true-delta_pred))

    print("Absolute diff in sigma:")
    print(np.abs(sigma_true-sigma_pred))    

    rmse_A = rmse(A_pred, A_true)
    rmse_delta = rmse(delta_true, delta_pred)
    rmse_sigma = rmse(sigma_true, sigma_pred) 

    return({"rmse_A": rmse_A, "rmse_delta": rmse_delta, "rmse_sigma": rmse_sigma})

## Experiment 0: MIRT-2PL Performance Benchmark

In [3]:
def mirt_performance_benchmark() -> dict:
    latent_dimension = 2
    item_dimension = 6
    sample_size=30

    #Define Population
    real_latent_cov = np.array([[1,0.2],
                    [0.2,1]])

    latent_distribution = multivariate_normal(mean=np.array([0,0]), cov=real_latent_cov)
    population = respondent_population(latent_dimension=latent_dimension, latent_distribution=latent_distribution)

    #Define Test
    A = np.array([[0.5,0],
                  [2,0],
                  [0,0.4],
                  [0.7,0.6],
                  [2, 0.9],
                  [0, 1.5]])
    Q = np.array([[1,0],
                  [1,0],
                  [0,1],
                  [1,1],
                  [1,1],
                  [0,1]])

    delta = np.array([0, 0.5, 1, 0, 1.5, 0.9])
    early_item_parameters = {"discrimination_matrix": A, "intercept_vector": delta, "q_matrix": Q, "item_dimension": item_dimension, "latent_dimension": latent_dimension}

    #Sample responses
    response_simulation_obj = response_simulation(population=population, early_item_params=early_item_parameters)
    sample = response_simulation_obj.sample(100)

    #Fit Parameters
    #Initialize model
    model = models.mirt_2pl(latent_dimension=2, item_dimension=6, A=Q, Q=Q, delta=np.ones(6), sigma=np.identity(2))
    e_step = em_algorithm.e_step_ga_mml(model=model)
    m_step = em_algorithm.m_step_ga_mml(model)
    em = em_algorithm.em_algo(e_step=e_step, m_step=m_step, model=model)

    #Fit Model
    start_time = time.time()
    em.fit(sample["early_responses"], max_iter=100)
    run_time =  (time.time() - start_time)

    #Measure Performance
    early_estimated_item_parameters = em.model.item_parameters
    early_estimated_person_parameters = em.model.person_parameters

    result_dict = {"sample": sample,
                   "real_early_parameters": {"item_parameters": early_item_parameters, "person_parameters": {"covariance": real_latent_cov}},
                   "estimated_early_parameters": {"item_parameters": early_estimated_item_parameters, "person_parameters": early_estimated_person_parameters},
                   "runtime": run_time,
                   "number_steps": em.n_steps}

    result_dict["early_performance"] = experiment_performance(estimated_parameter_dict=result_dict["real_early_parameters"], 
                                                              real_parameter_dict=result_dict["estimated_early_parameters"])


    return(result_dict)

In [4]:

def print_result(result_dict, description=""):
    print("------------------------------------")
    print("##### Results for {0}".format(description))
    print("Runtime: {0} seconds, {1} steps, {2} seconds per step \\".format(result_dict["runtime"], result_dict["number_steps"], result_dict["runtime"]/result_dict["number_steps"]))
    print("Performance: rmse-mean = {0}".format(np.mean(np.array(list(result_dict["early_performance"].values())))))

In [5]:
performance_dict = mirt_performance_benchmark()
#One Iteration: Runtime: 291 secs. 

EM Iteration 1
E-step
M-step
Maximize Q-0
Maximize the Q_i's
Step: 1: current parameter_diff: 1.8691908091913414, current marginal loglikelihood: -407.9292381189116
EM Iteration 2
E-step
M-step
Maximize Q-0
Maximize the Q_i's
Step: 2: current parameter_diff: 4.7431353432691274, current marginal loglikelihood: -423.59058334080146
EM Iteration 3
E-step
M-step
Maximize Q-0
Maximize the Q_i's
Step: 3: current parameter_diff: 4.242675132621676, current marginal loglikelihood: -426.38029949515936
EM Iteration 4
E-step
M-step
Maximize Q-0
Maximize the Q_i's
Step: 4: current parameter_diff: 2.107685163662153, current marginal loglikelihood: -445.85098510608395
EM Iteration 5
E-step
M-step
Maximize Q-0
Maximize the Q_i's
Step: 5: current parameter_diff: 5.480423589004223, current marginal loglikelihood: -433.8601820073843
EM Iteration 6
E-step
M-step
Maximize Q-0
Maximize the Q_i's
Step: 6: current parameter_diff: 2.7646296157581696, current marginal loglikelihood: -436.8688760327463
EM Iterati

c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\e_step_mirt_2pl.py:66: RuntimeWarning: divide by zero encountered in log
  factor = np.log(self.model.latent_density(theta, sigma=sigma))


Maximize the Q_i's
Step: 21: current parameter_diff: 3.8780256118120935, current marginal loglikelihood: -418.03751852057144
EM Iteration 22
E-step
M-step
Maximize Q-0
Maximize the Q_i's
Step: 22: current parameter_diff: 4.416581557435416, current marginal loglikelihood: -451.7184674649448
EM Iteration 23
E-step
M-step
Maximize Q-0
Maximize the Q_i's
Step: 23: current parameter_diff: 3.012524728368887, current marginal loglikelihood: -431.50549322896893
EM Iteration 24
E-step
M-step
Maximize Q-0
Maximize the Q_i's
Step: 24: current parameter_diff: 3.5620313281836125, current marginal loglikelihood: -443.2608026842842
EM Iteration 25
E-step
M-step
Maximize Q-0
Maximize the Q_i's
Step: 25: current parameter_diff: 2.8518102545469386, current marginal loglikelihood: -437.61512840411206
EM Iteration 26
E-step
M-step
Maximize Q-0
Maximize the Q_i's
Step: 26: current parameter_diff: 3.4213510097146194, current marginal loglikelihood: -468.26933211779743
EM Iteration 27
E-step
M-step
Maximize 

In [6]:
print_result(performance_dict, "More precise results through higher N and pop_size")

------------------------------------
##### Results for More precise results through higher N and pop_size
Runtime: 2224.739175796509 seconds, 101 steps, 22.02712055244068 seconds per step \
Performance: rmse-mean = 0.7503895962881408


------------------------------------
##### Results for Naive, all-Genetic, no vectorization
Runtime: 29100 seconds, 100+ steps, 291 seconds per step\
Performance: rmse-mean = 0.5896464054719196

------------------------------------
##### Results for Vectorized q_item
Runtime: 613.3871161937714 seconds, 100+ steps, 6.0731397642947655 seconds per step \
Performance: rmse-mean = 0.6466789259215839

------------------------------------
##### Results for More precise results through higher N and pop_size
Runtime: 2239.073846578598 seconds, 100+ steps, 22.169047985926714 seconds per step \
Performance: rmse-mean = 0.6384660709857508

## Procedure

1. Add Performance Benchmark (done)
2. Add Vectorization to q_item
3. Add better Initialization
3. Add Vectorization to q_0
4. Add q_0 Derivation 1. + BFGS
5. Add q_0 Derivation 2. + Newton Raphson
6. Use cython

## Experiment 1: MIRT-2PL Parameter Recovery

In [16]:
from unittest import result


def mirt_param_recovery(sample_size) -> dict:
    latent_dimension = 2
    item_dimension = 6
    sample_size=30

    #Define Population
    real_latent_cov = np.array([[1,0.2],
                    [0.2,1]])

    latent_distribution = multivariate_normal(mean=np.array([0,0]), cov=real_latent_cov)
    population = respondent_population(latent_dimension=latent_dimension, latent_distribution=latent_distribution)

    #Define Test
    A = np.array([[0.5,0],
                  [2,0],
                  [0,0.4],
                  [0.7,0.6],
                  [2, 0.9],
                  [0, 1.5]])
    Q = np.array([[1,0],
                  [1,0],
                  [0,1],
                  [1,1],
                  [1,1],
                  [0,1]])

    delta = np.array([0, 0.5, 1, 0, 1.5, 0.9])
    early_item_parameters = {"discrimination_matrix": A, "intercept_vector": delta, "q_matrix": Q, "item_dimension": item_dimension, "latent_dimension": latent_dimension}

    #Sample responses
    response_simulation_obj = response_simulation(population=population, early_item_params=early_item_parameters)
    sample = response_simulation_obj.sample(100)

    #Fit Parameters
    #Initialize model
    model = models.mirt_2pl(latent_dimension=2, item_dimension=6, A=Q, Q=Q, delta=np.ones(6), sigma=np.identity(2))
    e_step = em_algorithm.e_step_ga_mml(model=model)
    m_step = em_algorithm.m_step_ga_mml(model)
    em = em_algorithm.em_algo(e_step=e_step, m_step=m_step, model=model)

    #Fit Model
    start_time = time.time()
    em.fit(sample["early_responses"], max_iter=100)
    run_time =  (time.time() - start_time)

    #Measure Performance
    early_estimated_item_parameters = em.model.item_parameters
    early_estimated_person_parameters = em.model.person_parameters

    result_dict = {"sample": sample,
                   "real_early_parameters": {"item_parameters": early_item_parameters, "person_parameters": {"covariance": real_latent_cov}},
                   "estimated_early_parameters": {"item_parameters": early_estimated_item_parameters, "person_parameters": early_estimated_person_parameters},
                   "runtime": run_time,
                   "number_steps": em.n_steps}

    result_dict["early_performance"] = experiment_performance(estimated_parameter_dict=result_dict["real_early_parameters"], 
                                                              real_parameter_dict=result_dict["estimated_early_parameters"])


    return(result_dict)

In [3]:


#sample data
exper_dict = mirt_param_recovery(30)
U = exper_dict["sample"]["early_responses"]
Q = exper_dict["early_item_parameters"]["q_matrix"]

#Initialize model
model = models.mirt_2pl(latent_dimension=2, item_dimension=6, A=Q, Q=Q, delta=np.ones(6), sigma=np.identity(2))

e_step = em_algorithm.e_step_ga_mml(incomplete_data=U, model=model)
m_step = em_algorithm.m_step_ga_mml(model)

em = em_algorithm.em_algo(e_step=e_step, m_step=m_step, model=model)
em.fit(U, max_iter=5)


EM Iteration 1
E-step
M-step
Maximize Q-0
Highest Current Fitness:
-272.3020964947444
Length of Population = 20
Highest Current Fitness:
-272.3020964947444
Length of Population = 30
Maximize the Q_i's
Highest Current Fitness:
-80.38694365262204
Length of Population = 20
Highest Current Fitness:
-81.64548958485268
Length of Population = 20
Highest Current Fitness:
-70.8633188740732
Length of Population = 20
Highest Current Fitness:
-70.8633188740732
Length of Population = 30
Highest Current Fitness:
-80.60137913362004
Length of Population = 20
Highest Current Fitness:
-73.09757716674072
Length of Population = 20
Highest Current Fitness:
-72.2824475872277
Length of Population = 20
Highest Current Fitness:
-71.23025729049294
Length of Population = 30
Highest Current Fitness:
-71.23025729049294
Length of Population = 40
Step: 1: current parameter_diff: 7.507502483151055, current data likelihood: 0.0
EM Iteration 2
E-step
M-step
Maximize Q-0
Highest Current Fitness:
-273.9272087107773
Lengt

### Estimeted Parameters

In [5]:
experiment_performance(result_dict=exper_dict, em=em)

Absolute diff in A:
[[0.38755054 0.        ]
 [1.2584057  0.        ]
 [0.         0.94971913]
 [0.23538561 1.37671964]
 [1.84441689 0.19020049]
 [0.         0.66609164]]
Absolute diff in delta:
[1.07881578 1.45478874 0.17426022 0.67262895 0.08410975 1.02928687]
Absolute diff in sigma:
[[0.         0.04484618]
 [0.04484618 0.        ]]


{'rmse_A': 0.8400514908057599,
 'rmse_delta': 0.8971766856114028,
 'rmse_sigma': 0.03171104151355746}

In [20]:
experiment_performance(result_dict=exper_dict, em=em)

Absolute diff in A:
[[0.08534941 0.        ]
 [0.89466606 0.        ]
 [0.         0.5462264 ]
 [0.3758118  0.11807726]
 [1.         0.1       ]
 [0.         0.5       ]]
Absolute diff in delta:
[1.10062348 1.98836146 1.72083674 1.56568859 0.64138087 0.32523404]
Absolute diff in sigma:
[[0.         0.22172733]
 [0.22172733 0.        ]]


{'rmse_A': 0.45837138346516654,
 'rmse_delta': 1.3598259561775639,
 'rmse_sigma': 0.15678489631601697}

In [6]:
em.model.item_parameters

{'discrimination_matrix': array([[0.41465059, 0.        ],
        [1.10533394, 0.        ],
        [0.        , 0.9462264 ],
        [0.3241882 , 0.71807726],
        [1.        , 1.        ],
        [0.        , 1.        ]]),
 'intercept_vector': array([1.10062348, 2.48836146, 2.72083674, 1.56568859, 0.85861913,
        0.57476596]),
 'q_matrix': array([[1, 0],
        [1, 0],
        [0, 1],
        [1, 1],
        [1, 1],
        [0, 1]])}